In [18]:

# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings


In [19]:
df = pd.read_csv('data/clean_data.csv')

In [20]:
df.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7251 entries, 0 to 7250
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    7251 non-null   object 
 1   total_sqft  7251 non-null   float64
 2   bath        7251 non-null   float64
 3   price       7251 non-null   float64
 4   bhk         7251 non-null   int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 283.4+ KB


In [22]:
X = df.drop('price', axis=1)

In [23]:
X.head()

,location,total_sqft,bath,bhk
0,1st Block Jayanagar,2850.0,4.0,4
1,1st Block Jayanagar,1630.0,3.0,3
2,1st Block Jayanagar,1875.0,2.0,3
3,1st Block Jayanagar,1200.0,2.0,3
4,1st Block Jayanagar,1235.0,2.0,2


In [24]:
y = df['price']
y

0       428.0
1       194.0
2       235.0
3       130.0
4       148.0
        ...  
7246     70.0
7247    200.0
7248    110.0
7249     26.0
7250    400.0
Name: price, Length: 7251, dtype: float64

In [25]:

# Creating columns transformer with 3 type of transformers.
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

In [26]:
num_features

Index(['total_sqft', 'bath', 'bhk'], dtype='object')

In [27]:
cat_features

Index(['location'], dtype='object')

#### preprocessing

In [28]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_tranformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_tranformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features)
    ]
)

In [29]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(),
                                 Index(['location'], dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['total_sqft', 'bath', 'bhk'], dtype='object'))])

In [30]:
X= preprocessor.fit_transform(X)
X.shape

(7251, 245)

<7251x245 sparse matrix of type '<class 'numpy.float64'>'
	with 29004 stored elements in Compressed Sparse Row format>

In [34]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((5800, 245), (1451, 245))

In [35]:
def evaluate_model(true, prediction):
    mae = mean_absolute_error(true,prediction)
    mse = mean_squared_error(true,prediction)
    rmse = np.sqrt(mean_squared_error(true,prediction))
    r2_square = r2_score(true,prediction)
    return mae,mse,rmse,r2_square


In [36]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_mse,model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test,model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 33.8034
- Mean Absolute Error: 17.1126
- R2 Score: 0.8406
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 34.9336
- Mean Absolute Error: 17.9900
- R2 Score: 0.8746


Lasso
Model performance for Training set
- Root Mean Squared Error: 45.8779
- Mean Absolute Error: 23.1653
- R2 Score: 0.7065
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 49.8470
- Mean Absolute Error: 25.0757
- R2 Score: 0.7447


Ridge
Model performance for Training set
- Root Mean Squared Error: 34.0197
- Mean Absolute Error: 17.0988
- R2 Score: 0.8386
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 34.4150
- Mean Absolute Error: 17.8494
- R2 Score: 0.8783


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 32.1058
- Mean Absolute Error: 13.5861
- R2 Score: 0.8562
---------